# Processing and generating dataset for SPARK database

The database can be found [here](https://www.collaborativedrug.com/spark-data-downloads).

In [1]:
import os
import json
import pandas as pd
import numpy as np

from tqdm import tqdm

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option("display.max_columns", None)

In [2]:
DATA_DIR = "../data"

In [3]:
spark_data = []

for file in os.listdir(f"{DATA_DIR}/raw"):
    if file.startswith("SPARK"):
        df = pd.read_csv(
            f"{DATA_DIR}/raw/{file}",
            low_memory=False,
            on_bad_lines="skip",
        )
        df["source"] = file
        spark_data.append(df)
spark_df = pd.concat(spark_data, ignore_index=True)
spark_df.head()

,Compound Name,SMILES,Synonyms,SPARK Terms of Use,SPARK Data Downloads,Batch Compound-Batch ID,Compound ID - Source,SPARK Terms of Use and Data Downloads: SPARK Terms of Use,SPARK Terms of Use and Data Downloads: SPARK Data Downloads,Curated & Transformed MIC Data: Subset Person,Curated & Transformed MIC Data: Subset Conditions,Curated & Transformed MIC Data: MIC (µg/mL),Curated & Transformed MIC Data: Test Article Type,Curated & Transformed MIC Data: MIC (in µM) (µM),Curated & Transformed MIC Data: Threshold of growth inhibition,Curated & Transformed MIC Data: pMIC,Curated & Transformed MIC Data: Species,Curated & Transformed MIC Data: Strain,Curated & Transformed MIC Data: Strain - Other IDs,Curated & Transformed MIC Data: Clinical isolate?,Curated & Transformed MIC Data: Accumulation phenotype,Curated & Transformed MIC Data: Strain Genotype,Curated & Transformed MIC Data: Strain notes,Curated & Transformed MIC Data: Additives,Curated & Transformed MIC Data: Solvent,Curated & Transformed MIC Data: Solvent Amt. (%),Curated & Transformed MIC Data: Media,Curated & Transformed MIC Data: Method used,Curated & Transformed MIC Data: Source,Curated & Transformed MIC Data: Curation Date,Achaogen LpxC MIC Data: Subset Conditions,Achaogen LpxC MIC Data: MIC (ug/mL),Achaogen LpxC MIC Data: MIC/GIC reported as,Achaogen LpxC MIC Data: Test Article Type,Achaogen LpxC MIC Data: Threshold of growth inhibition,Achaogen LpxC MIC Data: MIC (in uM) (uM),Achaogen LpxC MIC Data: Species,Achaogen LpxC MIC Data: pMIC,Achaogen LpxC MIC Data: Strain,Achaogen LpxC MIC Data: Solvent,Achaogen LpxC MIC Data: Solvent Amt. (%),Achaogen LpxC MIC Data: Media,Achaogen LpxC MIC Data: Method used,Achaogen LpxC MIC Data: Source,Achaogen LpxC MIC Data: Curation Date,Curated & Transformed IC50 Data: Subset Conditions,Curated & Transformed IC50 Data: IC50 (nM),Curated & Transformed IC50 Data: IC50 (uM) (uM),Curated & Transformed IC50 Data: Test Article Type,Curated & Transformed IC50 Data: pIC50,Curated & Transformed IC50 Data: lower CI (nM),Curated & Transformed IC50 Data: upper CI (nM),Curated & Transformed IC50 Data: Target ID,Curated & Transformed IC50 Data: Molecular Target Name,Curated & Transformed IC50 Data: Molecular Target Synonym,Curated & Transformed IC50 Data: Species,Curated & Transformed IC50 Data: Target format,Curated & Transformed IC50 Data: Source,Curated & Transformed IC50 Data: Curation Date,Achaogen Protein Binding Data: Subset Conditions,Achaogen Protein Binding Data: Human Fraction Free (%),Achaogen Protein Binding Data: Human Fraction Free N,Achaogen Protein Binding Data: NHP (Cyno) Fraction Free (%),Achaogen Protein Binding Data: NHP (Cyno) Fraction Free (N),Achaogen Protein Binding Data: Dog (Beagle) Fraction Free (%),Achaogen Protein Binding Data: Dog (Beagle) N,Achaogen Protein Binding Data: Mouse Fraction Free (%),Achaogen Protein Binding Data: Mouse Fraction Free (N),Achaogen Protein Binding Data: Rat Fraction Free (%),Achaogen Protein Binding Data: Rat Fraction Free (N),Achaogen Protein Binding Data: Incubation Concentration,Achaogen Protein Binding Data: Incubation Concentration Units,Achaogen Protein Binding Data: Test Method,Achaogen Cytotox Data: Subset Conditions,Achaogen Cytotox Data: IC50 (uM),Achaogen Cytotox Data: IC50 Fit Error (uM),Achaogen Cytotox Data: R-Squared,Achaogen Cytotox Data: IC50 Std Dev (uM),Achaogen Cytotox Data: N,Achaogen LpxC MIC50/90 data: Subset Conditions,Achaogen LpxC MIC50/90 data: MIC50 (ug/mL),Achaogen LpxC MIC50/90 data: MIC90 (ug/mL),Achaogen LpxC MIC50/90 data: Min MIC (ug/mL),Achaogen LpxC MIC50/90 data: Max MIC (ug/mL),Achaogen LpxC MIC50/90 data: Secondary Panel Name,Achaogen Pae Clinical Isolate Panel Data: Subset Conditions,Achaogen Pae Clinical Isolate Panel Data: MIC5 (ug/mL),Achaogen LpxC PK data: Subset Conditions,Achaogen LpxC PK data: Analyte,Achaogen LpxC PK data: AUC 0-inf (hr*µg/mL),Achaogen LpxC PK data: AUC 0-t (hr*µg/mL),Achaogen LpxC PK data: C0 (µg/mL),Achaogen LpxC PK data: CL 

In [4]:
cols_to_keep = [
    "Compound Name",
    "SMILES",
    "PubMed ID",
    "DOI",
    "Curated & Transformed MIC Data: pMIC",
    "Curated & Transformed MIC Data: Species",
    "Achaogen LpxC MIC Data: Species",
    "Achaogen LpxC MIC Data: pMIC",
    "Curated & Transformed IC50 Data: pIC50",
    "Curated & Transformed IC50 Data: Species",
    "Extracted & Uploaded MIC Data: pMIC",
    "Extracted & Uploaded MIC Data: Species",
    "Extracted & Uploaded IC50 Data: pIC50",
    "Extracted & Uploaded IC50 Data: Species",
    "Porras 2020 Chem. Rev. XX:XXXX: pMIC",
    "Porras 2020 Chem. Rev. XX:XXXX: Species",
    "Chassagne 2020 Front. Pharmacol. 11:586548: MIC value (µg/mL)",
    "Chassagne 2020 Front. Pharmacol. 11:586548: Species",
]
spark_df = spark_df[cols_to_keep]

All the strain found in the list are bacerial, so no subsetting needed.

# Formatting data to be compliant with other sets

In [5]:
spark_df.replace("nan", np.nan, inplace=True)
spark_df.drop_duplicates(inplace=True)

In [6]:
spark_df.dropna(
    subset=[
        "Curated & Transformed MIC Data: Species",
        "Achaogen LpxC MIC Data: Species",
        "Extracted & Uploaded MIC Data: Species",
        "Porras 2020 Chem. Rev. XX:XXXX: Species",
        "Chassagne 2020 Front. Pharmacol. 11:586548: Species",
    ],
    how="all",
    inplace=True,
)

In [7]:
formatted_data = []

m = 0
for row in tqdm(spark_df.values):
    (
        cmp_name,
        smiles,
        pubmed_id,
        doi,
        curated_pmic,
        curated_pmic_species,
        achaogen_pmic_species,
        achaogen_pmic,
        curated_ic50,
        curated_ic50_species,
        extracted_pmic,
        extracted_pmic_species,
        extracted_ic50,
        extracted_ic50_species,
        porras_pmic,
        porras_pmic_species,
        chassagne_mic,
        chassagne_mic_species,
    ) = row

    species = [
        curated_pmic_species,
        achaogen_pmic_species,
        curated_ic50_species,
        extracted_pmic_species,
        extracted_ic50_species,
        porras_pmic_species,
        chassagne_mic_species,
    ]

    all_dtype_species = [type(i) for i in species]

    assert len(all_dtype_species) == 7

    empty_dtype_species = [
        k
        for i, k in enumerate(all_dtype_species)
        if k == float and np.isnan(species[i])
    ]

    # Fixing the assay values
    if isinstance(curated_pmic, str):
        if "<" in curated_pmic:
            curated_pmic = 0
        else:
            curated_pmic = curated_pmic.replace(">", "").replace("=", "")
    if isinstance(achaogen_pmic, str):
        if "<" in achaogen_pmic:
            achaogen_pmic = 0
        else:
            achaogen_pmic = achaogen_pmic.replace(">", "").replace("=", "")
    if isinstance(curated_ic50, str):
        if "<" in curated_ic50:
            curated_ic50 = 0
        else:
            curated_ic50 = curated_ic50.replace(">", "").replace("=", "")
    if isinstance(extracted_pmic, str):
        if "<" in extracted_pmic:
            extracted_pmic = 0
        else:
            extracted_pmic = extracted_pmic.replace(">", "").replace("=", "")

    # all empty
    if len(empty_dtype_species) == 7:
        continue

    # Cases where only one is present:
    elif len(empty_dtype_species) == 6:
        if (
            pd.notna(curated_ic50_species)
            and pd.isna(achaogen_pmic_species)
            and pd.isna(curated_pmic_species)
            and pd.isna(extracted_pmic_species)
            and pd.isna(extracted_ic50_species)
            and pd.isna(porras_pmic_species)
            and pd.isna(chassagne_mic_species)
        ):
            formatted_data.append(
                {
                    "cmp_name": cmp_name,
                    "smiles": smiles,
                    "species": curated_ic50_species,
                    "pIC50": float(curated_ic50),
                }
            )
        elif (
            pd.notna(achaogen_pmic_species)
            and pd.isna(curated_pmic_species)
            and pd.isna(curated_ic50_species)
            and pd.isna(extracted_pmic_species)
            and pd.isna(extracted_ic50_species)
            and pd.isna(porras_pmic_species)
            and pd.isna(chassagne_mic_species)
        ):
            formatted_data.append(
                {
                    "cmp_name": cmp_name,
                    "smiles": smiles,
                    "species": achaogen_pmic_species,
                    "pMIC": float(achaogen_pmic),
                }
            )

        elif (
            pd.notna(curated_pmic_species)
            and pd.isna(achaogen_pmic_species)
            and pd.isna(curated_ic50_species)
            and pd.isna(extracted_pmic_species)
            and pd.isna(extracted_ic50_species)
            and pd.isna(porras_pmic_species)
            and pd.isna(chassagne_mic_species)
        ):
            formatted_data.append(
                {
                    "cmp_name": cmp_name,
                    "smiles": smiles,
                    "species": curated_pmic_species,
                    "pMIC": float(curated_pmic),
                }
            )
        elif (
            pd.notna(extracted_pmic_species)
            and pd.isna(achaogen_pmic_species)
            and pd.isna(curated_ic50_species)
            and pd.isna(curated_pmic_species)
            and pd.isna(extracted_ic50_species)
            and pd.isna(porras_pmic_species)
            and pd.isna(chassagne_mic_species)
        ):
            formatted_data.append(
                {
                    "cmp_name": cmp_name,
                    "smiles": smiles,
                    "species": extracted_pmic_species,
                    "pMIC": float(extracted_pmic),
                }
            )
        elif (
            pd.notna(extracted_ic50_species)
            and pd.isna(achaogen_pmic_species)
            and pd.isna(curated_ic50_species)
            and pd.isna(curated_pmic_species)
            and pd.isna(extracted_pmic_species)
            and pd.isna(porras_pmic_species)
            and pd.isna(chassagne_mic_species)
        ):
            formatted_data.append(
                {
                    "cmp_name": cmp_name,
                    "smiles": smiles,
                    "species": extracted_ic50_species,
                    "pIC50": float(extracted_ic50),
                }
            )
        elif (
            pd.notna(porras_pmic_species)
            and pd.isna(achaogen_pmic_species)
            and pd.isna(curated_ic50_species)
            and pd.isna(curated_pmic_species)
            and pd.isna(extracted_pmic_species)
            and pd.isna(extracted_ic50_species)
            and pd.isna(chassagne_mic_species)
        ):
            formatted_data.append(
                {
                    "cmp_name": cmp_name,
                    "smiles": smiles,
                    "species": porras_pmic_species,
                    "pMIC": float(porras_pmic),
                }
            )
        elif (
            pd.notna(chassagne_mic_species)
            and pd.isna(achaogen_pmic_species)
            and pd.isna(curated_ic50_species)
            and pd.isna(curated_pmic_species)
            and pd.isna(extracted_pmic_species)
            and pd.isna(porras_pmic_species)
            and pd.isna(extracted_ic50_species)
        ):
            formatted_data.append(
                {
                    "cmp_name": cmp_name,
                    "smiles": smiles,
                    "species": chassagne_mic_species,
                    "pMIC": float(chassagne_mic),
                }
            )

    # Cases where two are present:
    elif len(empty_dtype_species) == 5:
        if pd.notna(curated_ic50_species) and pd.notna(curated_pmic_species):
            if curated_ic50_species == curated_pmic_species:
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": curated_ic50_species,
                        "pIC50": float(curated_ic50),
                        "pMIC": float(curated_pmic),
                    }
                )
            else:
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": curated_ic50_species,
                        "pIC50": float(curated_ic50),
                    }
                )
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": curated_pmic_species,
                        "pMIC": float(curated_pmic),
                    }
                )
        elif pd.notna(extracted_ic50_species) and pd.notna(extracted_pmic_species):
            if extracted_ic50_species == extracted_pmic_species:
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": extracted_pmic_species,
                        "pIC50": float(extracted_ic50),
                        "pMIC": float(extracted_pmic),
                    }
                )
            else:
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": extracted_ic50_species,
                        "pIC50": float(extracted_ic50),
                    }
                )
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": extracted_pmic_species,
                        "pMIC": float(extracted_pmic),
                    }
                )
        elif pd.notna(curated_ic50_species) and pd.notna(achaogen_pmic_species):
            if curated_ic50_species == achaogen_pmic_species:
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": achaogen_pmic_species,
                        "pIC50": float(curated_ic50),
                        "pMIC": float(achaogen_pmic),
                    }
                )
            else:
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": curated_ic50_species,
                        "pIC50": float(curated_ic50),
                    }
                )
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": achaogen_pmic_species,
                        "pMIC": float(achaogen_pmic),
                    }
                )
        elif pd.notna(curated_pmic_species) and pd.notna(achaogen_pmic_species):
            if curated_pmic_species == achaogen_pmic_species:
                if curated_pmic == 0:
                    val = float(achaogen_pmic)
                elif achaogen_pmic == 0:
                    val = float(curated_pmic)
                else:
                    val = (float(curated_pmic) + float(achaogen_pmic)) / 2
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": achaogen_pmic_species,
                        "pMIC": round(val, 2),
                    }
                )
            else:
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": curated_pmic_species,
                        "pMIC": float(curated_pmic),
                    }
                )
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": achaogen_pmic_species,
                        "pMIC": float(achaogen_pmic),
                    }
                )

    # Cases where three are present:
    elif len(empty_dtype_species) == 4:
        if (
            pd.notna(curated_ic50_species)
            and pd.notna(curated_pmic_species)
            and pd.notna(achaogen_pmic_species)
        ):
            if curated_pmic_species == achaogen_pmic_species:
                if curated_pmic == 0:
                    val = float(achaogen_pmic)
                elif achaogen_pmic == 0:
                    val = float(curated_pmic)
                else:
                    val = (float(curated_pmic) + float(achaogen_pmic)) / 2

            if curated_ic50_species == curated_pmic_species == achaogen_pmic_species:
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": achaogen_pmic_species,
                        "pMIC": round(val, 2),
                        "p150": float(curated_ic50),
                    }
                )

            elif curated_ic50_species == curated_pmic_species != achaogen_pmic_species:
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": curated_ic50_species,
                        "pIC50": float(curated_ic50),
                        "pMIC": float(curated_pmic),
                    }
                )
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": achaogen_pmic_species,
                        "pMIC": float(achaogen_pmic),
                    }
                )

            elif curated_ic50_species == achaogen_pmic_species != curated_pmic_species:
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": curated_ic50_species,
                        "pIC50": float(curated_ic50),
                        "pMIC": float(achaogen_pmic),
                    }
                )
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": curated_pmic_species,
                        "pMIC": float(curated_pmic),
                    }
                )

            elif curated_pmic_species != curated_ic50_species != achaogen_pmic_species:
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": curated_pmic_species,
                        "pMIC": float(curated_pmic),
                    }
                )
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": curated_ic50_species,
                        "pIC50": float(curated_ic50),
                    }
                )
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": achaogen_pmic_species,
                        "pMIC": float(achaogen_pmic),
                    }
                )

            elif curated_pmic_species == achaogen_pmic_species != curated_ic50_species:
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": achaogen_pmic_species,
                        "pMIC": round(val, 2),
                    }
                )
                formatted_data.append(
                    {
                        "cmp_name": cmp_name,
                        "smiles": smiles,
                        "species": curated_ic50_species,
                        "pIC50": float(curated_ic50),
                    }
                )

100%|██████████| 126583/126583 [00:02<00:00, 45199.82it/s]


In [8]:
df = pd.DataFrame(formatted_data)
df.head()

,cmp_name,smiles,species,pMIC,pIC50,p150
0,SPK-0125760,NaN,Pseudomonas aeruginosa,4.7,NaN,NaN
1,SPK-0125760,NaN,Pseudomonas aeruginosa,4.1,NaN,NaN
2,SPK-0125760,NaN,Klebsiella pneumoniae,0.0,NaN,NaN
3,SPK-0125760,NaN,Klebsiella pneumoniae,0.0,NaN,NaN
4,SPK-0125760,NaN,Escherichia coli,6.8,NaN,NaN


# Matching bacterial to gram strains

In [9]:
bacterial_species = json.load(open(f"{DATA_DIR}/mapping/bact_mapper.json"))
len(bacterial_species)

382

In [10]:
df["strain_type"] = df["species"].map(bacterial_species)

In [11]:
df.to_csv(f"{DATA_DIR}/processed/bacterial_spark.tsv", sep="\t", index=False)

In [12]:
df["strain_type"].value_counts()

strain_type
gram-negative    71059
gram-positive     8487
acid-fast          343
Name: count, dtype: int64

In [13]:
df["species"].nunique()

271